In [1]:
import torch
print("Is GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")

Is GPU Available: True
GPU Name: Tesla V100-SXM2-32GB


In [5]:
import torch
import os
# from data_loading import RegressionTaskData
from network import CNNRegression
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil

from typing import Tuple
import torch.nn as nn
import numpy as np
from torch.utils.tensorboard import SummaryWriter

import torchvision
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

import seaborn as sns

ModuleNotFoundError: No module named 'network'